In [1]:
import numpy as np

In [57]:
class regular_linear_regression:
    def __init__(self,train_size,test_size,lam):
        self.train_size=train_size
        self.test_size=test_size
        self.lam=lam
        
    def read_train_set(self):
        raw_train=open("hw4_train.dat")
        X=np.zeros((self.train_size,3))
        Y=np.zeros((self.train_size,1))
        i=0
        for line in raw_train:
            X[i,0]=1
            X[i,1]=line.split(" ")[0]
            X[i,2]=line.split(" ")[1]
            Y[i,0]=line.split(" ")[2].strip()
            i+=1
        return X,Y
    
    def read_test_set(self):
        raw_test=open("hw4_test.dat")
        X=np.zeros((self.test_size,3))
        Y=np.zeros((self.test_size,1))
        i=0
        for line in raw_test:
            X[i,0]=1
            X[i,1]=line.split(" ")[0]
            X[i,2]=line.split(" ")[1]
            Y[i,0]=line.split(" ")[2].strip()
            i+=1
        return X,Y
    
    def get_w(self,X,Y):
        a=np.eye(3)
        a[0,0]=0               #注意，这个最好加上，尽管在台大教程里没说要加上，加上之后误差更小
        return np.linalg.inv(np.dot(X.T,X)+self.lam*a).dot(X.T).dot(Y)
    
    def error(self,X,Y,w):
        Y_cal=np.dot(X,w)
        sum_error=sum(np.sign(Y_cal)!=Y)
        return sum_error/np.size(Y)


#13
if __name__ == '__main__':
    lin_model=regular_linear_regression(200,1000,10)
    X_train,Y_train=lin_model.read_train_set()
    X_test,Y_test=lin_model.read_test_set()
    w=lin_model.get_w(X_train,Y_train)
    Ein=lin_model.error(X_train,Y_train,w)
    Eout=lin_model.error(X_test,Y_test,w)
    print(w)
    print(Ein)
    print(Eout)
    
        
        

[[-0.93238149]
 [ 1.04618645]
 [ 1.046171  ]]
[0.05]
[0.045]


In [66]:
#14     用Ein最小来选择lam
Ein_min=10
lam_best=0
Eout_corres=0
for a in range(-10,3):
    lin_model=regular_linear_regression(200,1000,pow(10,a))
    X_train,Y_train=lin_model.read_train_set()
    w=lin_model.get_w(X_train,Y_train)
    Ein=lin_model.error(X_train,Y_train,w)
    Eout=lin_model.error(X_test,Y_test,w)
    if Ein<=Ein_min:
        Ein_min=Ein
        lam_best=pow(10,a)
        Eout_corres=Eout
print(lam_best,Ein_min,Eout_corres)

1e-08 [0.015] [0.02]


In [67]:
#15       用Eout最小来选择lam
Eout_min=10
lam_best=0
Ein_corres=0
for a in range(-10,3):
    lin_model=regular_linear_regression(200,1000,pow(10,a))
    X_train,Y_train=lin_model.read_train_set()
    w=lin_model.get_w(X_train,Y_train)
    Ein=lin_model.error(X_train,Y_train,w)
    Eout=lin_model.error(X_test,Y_test,w)
    if Eout<=Eout_min:
        Ein_corres=Ein
        lam_best=pow(10,a)
        Eout_min=Eout
print(lam_best,Ein_corres,Eout_min)

1e-07 [0.03] [0.015]


In [70]:
#16        训练整集分成了训练集和验证集，用训练集误差最小来选择lam
Etrain_min=10
lam_best=0
Eval_corres=0
Eout_corres=0
for a in range(-10,3):
    lin_model=regular_linear_regression(200,1000,pow(10,a))
    X_train,Y_train=lin_model.read_train_set()
    w=lin_model.get_w(X_train[:120,:],Y_train[:120,:])
    Etrain=lin_model.error(X_train[:120,:],Y_train[:120,:],w)
    Eval=lin_model.error(X_train[120:,:],Y_train[120:,:],w)
    Eout=lin_model.error(X_test,Y_test,w)
    if Etrain<=Etrain_min:
        Etrain_min=Etrain
        lam_best=pow(10,a)
        Eval_corres=Eval
        Eout_corres=Eout
print(lam_best,Etrain_min,Eval_corres,Eout_corres)

1e-08 [0.] [0.05] [0.025]


In [72]:
#17       训练整集分成了训练集和验证集，用验证集误差最小来选择lam
Etrain_corres=0
lam_best=0
Eval_min=10
Eout_corres=0
for a in range(-10,3):
    lin_model=regular_linear_regression(200,1000,pow(10,a))
    X_train,Y_train=lin_model.read_train_set()
    w=lin_model.get_w(X_train[:120,:],Y_train[:120,:])
    Etrain=lin_model.error(X_train[:120,:],Y_train[:120,:],w)
    Eval=lin_model.error(X_train[120:,:],Y_train[120:,:],w)
    Eout=lin_model.error(X_test,Y_test,w)
    if Eval<=Eval_min:
        Eval_min=Eval
        lam_best=pow(10,a)
        Etrain_corres=Etrain
        Eout_corres=Eout
print(lam_best,Etrain_corres,Eval_min,Eout_corres)

1 [0.03333333] [0.0375] [0.028]


In [74]:
#18  利用17的lambda
lam=1
lin_model=regular_linear_regression(200,1000,1)
X_train,Y_train=lin_model.read_train_set()
w=lin_model.get_w(X_train,Y_train)
Ein=lin_model.error(X_train,Y_train,w)
Eout=lin_model.error(X_test,Y_test,w)
print(Ein,Eout)

[0.035] [0.02]


In [91]:
#19
fold=40       #K-fold交叉验证
Eval_min=10
lam_best=0
for a in range(-10,3):
    lin_model=regular_linear_regression(200,1000,pow(10,a))
    X_train,Y_train=lin_model.read_train_set()
    Eval_tot=0
    for i in range(5):
        w=lin_model.get_w(np.concatenate((X_train[(i+1)*40:200,:],X_train[0:i*40,:])),np.concatenate((Y_train[(i+1)*40:200,:],Y_train[0:i*40,:])))
        Eval=lin_model.error(X_train[i*40:(i+1)*40,:],Y_train[i*40:(i+1)*40,:],w)
        Eval_tot=Eval_tot+Eval
    Eval_avg=Eval_tot/5
    if(Eval_avg<Eval_min):
        lam_best=pow(10,a)
        Eval_min=Eval_avg
print(lam_best,Eval_min)

1e-08 [0.03]


In [93]:
#20  利用19交叉验证得到的lambda (发现，这个交叉验证得到的lam应该是最准确的)
lam=1e-08
lin_model=regular_linear_regression(200,1000,lam)
X_train,Y_train=lin_model.read_train_set()
w=lin_model.get_w(X_train,Y_train)
Ein=lin_model.error(X_train,Y_train,w)
Eout=lin_model.error(X_test,Y_test,w)
print(Ein,Eout)

[0.015] [0.02]
